In [2]:
!pip install f5_tts

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00a 0:00:01


In [3]:
import torch
import argparse
from safetensors.torch import load_file
from f5_tts.model.utils import list_str_to_idx, get_tokenizer

In [8]:
def inspect_lora_weights(sd):
    # find all LoRA params in the after-finetuned checkpoint
    lora_items = {k: v for k,v in sd.items() if ".lora_" in k and isinstance(v, torch.Tensor)}
    if not lora_items:
        print("No LoRA weights found in this checkpoint.")
        return

    print("LoRA adapter parameter stats:\n")
    total_norm2 = 0.0
    total_elements = 0
    for name, tensor in sorted(lora_items.items()):
        t = tensor.cpu()
        mean_abs = t.abs().mean().item()
        max_abs  = t.abs().max().item()
        norm2    = t.norm().item()
        total_norm2 += norm2**2
        total_elements += t.numel()
        print(f"{name:60s}  shape={tuple(t.shape)}  mean|·|={mean_abs:.3e} max|·|={max_abs:.3e} ‖·‖₂={norm2:.3e}")

    overall_fro = total_norm2**0.5
    avg_norm    = overall_fro / (total_elements**0.5)
    print("\nSummary:")
    print(f"  Total LoRA parameters : {total_elements:,}")
    print(f"  Combined Frobenius ‖Δ‖₂ : {overall_fro:.3e}")
    print(f"  Avg per-param scale    : {avg_norm:.3e}")
    print()

In [10]:
# Path to your checkpoints
before_ckpt = "/work/users/r/p/rphadke/JSALT/ckpts/pretrained_model_1250000.safetensors"
after_ckpt = "/work/users/r/p/rphadke/JSALT/ckpts/fisher_chunks_1K_LoRAv3.3/model_75000.pt"
tok_path = "/work/users/r/p/rphadke/JSALT/fisher_chunks_0.1K_v2/vocab.txt"
vocab_char_map, vocab_size = get_tokenizer(tok_path, "custom")

# ID of your new speaker token (e.g. tokenizer.convert_tokens_to_ids("<spk>"))
spk_chg_token = "-"

    # 1) Load state dicts
sd_before = load_file(before_ckpt, device="cpu")
print("\n == Prior Checkpoint ==")
# print(set(sd_before.keys()))
sd_after  = torch.load(after_ckpt,  map_location="cpu")["model_state_dict"]
sd_after = {k.replace("ema_model.", ""): v for k, v in sd_after.items()}
print("\n == New Checkpoint ==")
# print(set(sd_after.keys()))


 == Prior Checkpoint ==


/tmp/ipykernel_1548620/732349108.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd_after  = torch.load(after_ckpt,  map_location="cpu")["model_state_dict"]



 == New Checkpoint ==


In [15]:
# 2) Extract embedding weights
#    adjust the key if your checkpoint nests it differently
Wb = sd_before["ema_model.transformer.text_embed.text_embed.weight"]
Wa = sd_after["base_model.model.transformer.text_embed.text_embed.weight"]

ib = sd_before["ema_model.transformer.input_embed.proj.weight"]
ia = sd_after["base_model.model.transformer.input_embed.proj.weight"]

# 3) Compute absolute difference
delta = (Wa - Wb).abs()

idelta = (ib - ia).abs()

# 4) Metrics
speaker_chg_token_idx = list_str_to_idx([spk_chg_token], vocab_char_map=vocab_char_map)

max_overall   = delta.max().item()
max_speaker   = delta[speaker_chg_token_idx].max().item()

print("\n== Text Embedding Drift ==")
print(f"→ Max text embedding matrix change |Δ| overall:                  {max_overall:.6f}")
print(f"→ Max |Δ| on token {speaker_chg_token_idx}: {max_speaker:.6f}")

print("\n== Input Embedding Drift ==")
print(f"→ Mean input embedding weight change |Δ| overall:                  {idelta.max().item():.6f}")

print("== LoRA Weight Scales ==")
inspect_lora_weights(sd_after)


== Text Embedding Drift ==
→ Mean text embedding matrix change |Δ| overall:                  0.000106
→ Max |Δ| on token tensor([[13]]): 0.000003

== Input Embedding Drift ==
→ Mean input embedding weight change |Δ| overall:                  0.000918
== LoRA Weight Scales ==
LoRA adapter parameter stats:

base_model.model.transformer.text_embed.text_blocks.0.pwconv1.lora_A.default.weight  shape=(64, 512)  mean|·|=2.220e-02 max|·|=4.427e-02 ‖·‖₂=4.636e+00
base_model.model.transformer.text_embed.text_blocks.0.pwconv1.lora_B.default.weight  shape=(1024, 64)  mean|·|=9.179e-06 max|·|=8.920e-05 ‖·‖₂=3.179e-03
base_model.model.transformer.text_embed.text_blocks.0.pwconv2.lora_A.default.weight  shape=(64, 1024)  mean|·|=1.567e-02 max|·|=3.138e-02 ‖·‖₂=4.627e+00
base_model.model.transformer.text_embed.text_blocks.0.pwconv2.lora_B.default.weight  shape=(512, 64)  mean|·|=1.286e-05 max|·|=1.160e-04 ‖·‖₂=3.125e-03
base_model.model.transformer.text_embed.text_blocks.1.pwconv1.lora_A.default.weigh